# TinyLlama 1.1B Chat: From GGUF to Interactive Inference

Learn how to load the TinyLlama‑1.1B‑Chat‑v1.0 GGUF model from Hugging Face, run fast inference, customize prompts, and deploy a lightweight chat API. This lesson covers environment setup, model loading, generation, batching, and basic fine‑tuning using the Hugging Face ecosystem.

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Install and configure the required Python packages for GGUF inference.
2. Load the TinyLlama‑1.1B‑Chat model and tokenizer from the Hugging Face Hub.
3. Generate text with custom prompts and control generation parameters.
4. Deploy the model as a simple REST API for real‑time chat.


## Prerequisites

- Basic Python programming (3.8+).
- Familiarity with Hugging Face Transformers API.


## Setup

Let's install the required packages and set up our environment.


In [ ]:
# Create requirements.txt
requirements = '''transformers==4.35.0
torch==2.0.1
accelerate==0.21.0
huggingface_hub==0.20.3
fastapi==0.110.0
uvicorn==0.29.0'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print('✅ requirements.txt created!')

## Step 1: Environment & Dependencies

Welcome to the first step of your TinyLlama adventure! Think of this section as setting up a tiny kitchen before you start cooking a fancy dish. We’ll install the right tools (Python packages), make sure your computer can talk to the Hugging Face Hub, and set up a safe place to keep your credentials.

**Why do we need all this?**
- `transformers` gives us the recipe for TinyLlama.
- `torch` is the engine that runs the math.
- `accelerate` helps us use your GPU (if you have one) or CPU efficiently.
- `huggingface_hub` lets us download the GGUF file.
- `fastapi` and `uvicorn` are for the future chat API.

We’ll also show you how to create a virtual environment so your project stays tidy.


In [ ]:
# 1️⃣ Create a virtual environment (recommended)
# This keeps your project dependencies isolated.
# If you already have one, skip to the pip install step.

import subprocess, sys

try:
    subprocess.check_call([sys.executable, "-m", "venv", "venv"])
    print("✅ Virtual environment 'venv' created.")
except Exception as e:
    print("⚠️  Virtual environment creation failed:", e)

# 2️⃣ Activate the environment (Unix/macOS)
#    source venv/bin/activate
#    (Windows) venv\Scripts\activate.bat

# 3️⃣ Install the required packages
# We pin exact versions for reproducibility.

requirements = [
    "transformers==4.35.0",
    "torch==2.0.1",
    "accelerate==0.21.0",
    "huggingface_hub==0.20.3",
    "fastapi==0.110.0",
    "uvicorn==0.29.0"
]

try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", *requirements])
    print("✅ All packages installed successfully.")
except subprocess.CalledProcessError as e:
    print("⚠️  Package installation failed. Try running pip manually.")
    sys.exit(1)


## 4️⃣ Set up your Hugging Face token

If you’re downloading a private model, you’ll need a **HF_TOKEN**. For public models like TinyLlama‑1.1B‑Chat‑v1.0, it’s optional, but it speeds up downloads.

**How to get one:**
1. Sign in to [Hugging Face](https://huggingface.co/).
2. Go to *Settings → Access Tokens*.
3. Click *New token*, give it a name, and copy the value.

**Why store it in an environment variable?**
- Keeps it out of your code.
- Lets `huggingface_hub` automatically pick it up.


In [ ]:
# Export the token for Unix/macOS
# Replace YOUR_HF_TOKEN with the string you copied.
# You can add this line to ~/.bashrc or ~/.zshrc for persistence.

import os

os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN"  # <-- replace this
print("✅ HF_TOKEN set in environment (for this session).")

# For Windows PowerShell:
#   $env:HF_TOKEN = "YOUR_HF_TOKEN"


## 🎉 You’re all set!

Run the cells above in a Jupyter notebook or a Python script. If everything prints `✅`, you’re ready to load TinyLlama in the next section. If you hit any errors, double‑check:
- Python version (`python --version` should be 3.8+).
- Internet connection.
- Correct spelling of package names.

Happy coding! 🚀

## Step 2: Understanding GGUF and TinyLlama

Before we can load TinyLlama, let’s demystify two key concepts:

1. **GGUF** – Think of a GGUF file as a *tiny, super‑compressed cookbook*. It stores the model’s weights in a binary format that’s smaller than the usual `.bin` or `.pt` files, but still contains all the information the model needs to talk.
2. **TinyLlama‑1.1B‑Chat** – This is a lightweight version of the Llama family, trimmed down to 1.1 billion parameters. It’s like a compact smartphone that still runs most apps, but uses less memory and CPU.

Why GGUF? The main benefits are:
- **Speed** – Loading is faster because the file is smaller.
- **Memory** – Less RAM is required to keep the model in memory.
- **Portability** – You can ship the model to edge devices or cloud functions with minimal bandwidth.

Below we’ll inspect the GGUF file, check its size, and confirm that the Hugging Face Hub can handle it.


In [ ]:
# 1️⃣ Inspect the GGUF file size and metadata
# We’ll use the `huggingface_hub` library to download the file locally
# and then read a few bytes to confirm it’s a GGUF file.

import os
from huggingface_hub import hf_hub_download

# Path where the GGUF will be stored
model_id = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
filename = "TinyLlama-1.1B-Chat-v1.0.Q4_K_M.gguf"  # one of the quantized variants

# Download (or use cached copy)
local_path = hf_hub_download(repo_id=model_id, filename=filename)
print(f"✅ GGUF file downloaded to: {local_path}")

# Show file size in megabytes
size_mb = os.path.getsize(local_path) / (1024 ** 2)
print(f"📦 File size: {size_mb:.2f} MB")

# Peek at the first few bytes – GGUF files start with the ASCII string 'GGUF'
with open(local_path, "rb") as f:
    header = f.read(4)
print(f"🗂️ Header bytes: {header}")


### What we just did
- **`hf_hub_download`** pulls the file from Hugging Face and caches it locally.
- We printed the file size to see how lightweight it is.
- The header check confirms the file is indeed a GGUF binary.

If you see `b'GGUF'` in the header, you’re good to go!


In [ ]:
# 2️⃣ Quick sanity check: load the model with transformers
# This will load the GGUF file into memory. It may take a few seconds.
# We’ll use the `trust_remote_code=True` flag to allow the model class to be loaded.

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load tokenizer – TinyLlama uses the Llama tokenizer
print("🔄 Loading tokenizer…")
model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
# The tokenizer is the same as the original Llama model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
print("✅ Tokenizer loaded.")

# Load the GGUF model
print("🔄 Loading GGUF model…")
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
print("✅ GGUF model loaded.")


### TL;DR
- GGUF is a compact, binary format for model weights.
- TinyLlama‑1.1B‑Chat is a 1.1 billion‑parameter Llama variant, great for quick inference.
- The Hugging Face Hub and `transformers` make it trivial to download and load these files.

In the next section we’ll actually generate text with this model.


## Step 3: Loading the Model & Tokenizer

Now that we’ve downloaded the TinyLlama GGUF file, it’s time to bring it into memory. Think of the model as a *recipe book* and the tokenizer as the *chef’s measuring spoon*. The tokenizer turns your words into numbers the model can understand, and the model uses those numbers to predict the next word.

We’ll load both components with a single line each, using the `transformers` library. The `trust_remote_code=True` flag tells Hugging Face that it’s okay to run the custom code that ships with TinyLlama.

If you’re on a machine with a GPU, the model will automatically use it. If not, it will fall back to the CPU.

Let’s get started!


In [ ]:
# 1️⃣ Load the tokenizer
# The tokenizer is the same as the original Llama tokenizer
# It converts text to token ids and back.

from transformers import AutoTokenizer

model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"

try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True,
        use_fast=True  # fast tokenizers are a bit faster
    )
    print("✅ Tokenizer loaded successfully.")
except Exception as e:
    print("⚠️  Failed to load tokenizer:", e)
    raise

# Quick sanity check: encode a simple sentence
sample = "Hello, TinyLlama!"
encoded = tokenizer.encode(sample, add_special_tokens=True)
print("🔢 Token IDs:", encoded)
print("🧩 Decoded back:", tokenizer.decode(encoded))


### What just happened?
- `AutoTokenizer.from_pretrained` pulls the tokenizer files from the Hugging Face Hub.
- `add_special_tokens=True` adds the beginning‑of‑sentence token that TinyLlama expects.
- The quick encode/decode round‑trip confirms the tokenizer works.

Now we’re ready to load the heavy‑lifting part: the model itself.


In [ ]:
# 2️⃣ Load the GGUF model
# This step can take a few seconds, especially on a CPU.
# If you have a GPU, the model will automatically use it.

from transformers import AutoModelForCausalLM
import torch

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        device_map="auto",  # let accelerate decide where to place layers
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    print("✅ GGUF model loaded successfully.")
except Exception as e:
    print("⚠️  Failed to load model:", e)
    raise

# Verify that the model can run a quick forward pass
input_ids = tokenizer.encode("Tell me a joke.", return_tensors="pt")
if torch.cuda.is_available():
    input_ids = input_ids.to("cuda")

with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=20)

print("🤖 Model output:", tokenizer.decode(outputs[0], skip_special_tokens=True))


### Why `device_map="auto"`?
- On a GPU, the model’s layers are split across memory to avoid overflow.
- On a CPU, it keeps everything in RAM but still uses the most efficient layout.

If you want to force the model onto a specific device, you can replace `device_map="auto"` with `device_map="cpu"` or `device_map="cuda:0"`.


## Step 4: Basic Text Generation

In this step we’ll actually ask TinyLlama‑1.1B‑Chat to write something. Think of the model as a very clever robot that has read a huge book. When you give it a *prompt* (the first few words or a question), it predicts the next word, then the next, and so on, until it reaches a stopping point.

We’ll cover:

1. **Setting a random seed** – so you can reproduce the same answer.
2. **Choosing generation parameters** – how long the answer is, how creative it is, etc.
3. **Running the generation** – with a tiny code snippet.
4. **Decoding the token IDs back to text** – turning numbers into readable words.

All code is split into small cells so you can run them one by one and see the output immediately.


In [ ]:
# 1️⃣  Set a reproducible random seed
# This makes sure that every time you run the notebook you get the same "random" words.
# If you want a different answer, change the number.
import random
import torch

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
# If you have a GPU, also set its seed for full reproducibility
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"Random seed set to {SEED}")

⚠️ **Warning**: Even with a fixed seed, the model may still produce slightly different outputs on different hardware or library versions. For strict reproducibility, pin the exact library versions as listed in `requirements.txt`.

In [ ]:
# 2️⃣  Choose generation parameters
# These control how the model behaves:
#   - max_new_tokens: how many words (roughly) to generate
#   - temperature: 0 = deterministic, higher = more random
#   - top_p (nucleus sampling): keep the top p% of probability mass
#   - top_k: keep only the top k most likely next words
#   - do_sample: whether to sample or take the most likely word

generation_kwargs = {
    "max_new_tokens": 150,
    "temperature": 0.7,
    "top_p": 0.9,
    "top_k": 50,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,  # avoid warnings
}
print("Generation parameters:", generation_kwargs)


💡 **Tip**: Lower `temperature` (e.g., 0.3) for more factual, deterministic answers. Raise it (e.g., 1.0) for creative or whimsical responses.

In [ ]:
# 3️⃣  Run the generation
# We’ll use a simple prompt and let the model finish the sentence.
prompt = "Once upon a time in a quiet village, a mysterious traveler arrived"

# Encode the prompt into token IDs
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
input_ids = input_ids.to(device)

# Generate
generated_ids = model.generate(input_ids, **generation_kwargs)

# Decode the output (skip the prompt part)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("\nGenerated text:\n")
print(generated_text)


📝 **Note**: The `skip_special_tokens=True` flag removes invisible tokens like `<eos>` that the model uses internally. If you want to see the raw token IDs, set it to `False`.

## Quick sanity check

If the output looks garbled or the model hangs, try:

1. **Restart the kernel** – sometimes GPU memory gets stuck.
2. **Reduce `max_new_tokens`** – a very long generation can exhaust memory.
3. **Check the model name** – ensure you loaded the correct GGUF file.

Happy generating!

## Step 5: Prompt Engineering & Generation Controls

Now that you can get TinyLlama to write something, let’s learn how to *guide* it. Think of the model as a chef: the prompt is the recipe, and the generation parameters are the cooking settings (heat, time, seasoning). By tweaking these, you can make the output more factual, more creative, or just shorter.

We’ll cover:

1. **Prompt structure** – how to format the prompt for chat.
2. **Temperature & sampling** – control randomness.
3. **Top‑p / Top‑k** – narrow the word choices.
4. **Repetition penalty & length penalty** – avoid loops.
5. **Stopping sequences** – tell the model when to stop.

All code snippets are tiny and ready to run.


In [ ]:
# 1️⃣  Build a chat‑style prompt
# TinyLlama expects a conversation format:
# "User: <your question>\nAssistant:"  
# This helps the model understand the role.

user_question = "Explain quantum entanglement in simple terms."
prompt = f"User: {user_question}\nAssistant:"
print("Prompt:\n", prompt)


💡 **Tip**: Adding a short instruction before the user question (e.g., "You are a friendly teacher.") can nudge the style.


In [ ]:
# 2️⃣  Temperature & sampling
# Temperature: 0 = deterministic, 1 = fully random
# Do_sample: True means the model will sample from the distribution.

generation_kwargs = {
    "max_new_tokens": 200,
    "temperature": 0.5,  # moderate creativity
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}
print("Generation kwargs (temp=0.5):", generation_kwargs)


⚠️ **Warning**: Very low temperature (e.g., 0) can make the model repeat the same phrase over and over.


In [ ]:
# 3️⃣  Top‑p (nucleus) and Top‑k sampling
# These limit the pool of next‑word candidates.

generation_kwargs.update({
    "top_p": 0.9,   # keep the top 90% probability mass
    "top_k": 50,    # keep only the 50 most likely words
})
print("Updated kwargs (top_p=0.9, top_k=50):", generation_kwargs)


📝 **Note**: Setting both top_p and top_k is fine; the model will apply the stricter of the two.


In [ ]:
# 4️⃣  Repetition & length penalties
# These discourage the model from looping or being too short.

generation_kwargs.update({
    "repetition_penalty": 1.2,  # >1 discourages repeats
    "length_penalty": 1.0,      # >1 favors longer outputs
})
print("Final kwargs:", generation_kwargs)


💡 **Tip**: If the model keeps saying "Sure" or "Here is", increase the repetition_penalty.


In [ ]:
# 5️⃣  Stopping sequences
# Tell the model to stop when it sees a certain token.
# For chat, we often stop at a new "User:" prompt.

generation_kwargs.update({
    "stop_strings": ["\nUser:"]
})
print("With stop_strings:", generation_kwargs)


## Run the tuned generation

```python
# Encode prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(device)

# Generate
generated_ids = model.generate(input_ids, **generation_kwargs)

# Decode, skipping the prompt part
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("\nGenerated answer:\n", generated_text)
```


If you want to experiment, change one parameter at a time and compare the outputs. That’s the essence of prompt engineering!


## Step 6: Batch Inference for Performance

Content for Step 6: Batch Inference for Performance will be added here.

## Step 7: Fine‑Tuning on a Tiny Dataset

Content for Step 7: Fine‑Tuning on a Tiny Dataset will be added here.

## 8️⃣ Deploying TinyLlama as a FastAPI Chat Service

In the previous steps we learned how to generate text locally. Now we’ll expose that capability as a tiny, production‑ready web service using **FastAPI**. Think of FastAPI as a friendly waiter that takes a customer’s request (the prompt), asks the chef (TinyLlama) for a dish (the answer), and brings it back in a nicely formatted JSON.

We’ll cover:

1. **Environment setup** – install FastAPI and Uvicorn.
2. **Model loading** – load the GGUF model once at startup.
3. **Endpoint design** – a `/chat` POST endpoint that accepts a prompt.
4. **Streaming (optional)** – how to stream tokens back to the client.
5. **Running locally** – launch the server with `uvicorn`.
6. **Testing** – a quick `curl` example.

All code is split into small cells so you can copy‑paste and run them one by one.


In [ ]:
# 1️⃣  Install FastAPI & Uvicorn (run once)
# pip install fastapi uvicorn


⚠️ **Warning**: If you’re on Windows, you may need to run `pip install uvicorn[standard]` to get the ASGI server working.


In [ ]:
# 2️⃣  Import libraries and set a reproducible seed
import os
import random
import torch
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Optional: use a Hugging Face token for private models
HF_TOKEN = os.getenv("HF_TOKEN", None)


💡 **Tip**: Store your `HF_TOKEN` in a `.env` file and load it with `python-dotenv` for convenience.


In [ ]:
# 3️⃣  Load the TinyLlama GGUF model once at startup
MODEL_ID = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"

# The GGUF file is automatically downloaded by the hub
# `trust_remote_code=True` allows the model to load custom code
print("Loading tokenizer and model…")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=False,
    trust_remote_code=True,
    token=HF_TOKEN,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",  # let transformers pick GPU/CPU
    trust_remote_code=True,
    token=HF_TOKEN,
)
model.eval()  # disable dropout
print("Model loaded on", next(model.parameters()).device)


⚠️ **Warning**: Loading the model can take 30–60 s on a laptop. Keep the process running; the server will start only after the model is ready.


## 4️⃣  Define the FastAPI app and request schema

We’ll create a simple Pydantic model for the request body. The client sends a JSON object with a `prompt` field, and optionally a `max_new_tokens` field.


In [ ]:
class ChatRequest(BaseModel):
    prompt: str
    max_new_tokens: int | None = 150

app = FastAPI(title="TinyLlama Chat API", description="Generate chat responses with TinyLlama 1.1B")


## 5️⃣  Create the `/chat` endpoint

The endpoint will:

1. Encode the prompt.
2. Run inference inside `torch.no_grad()` to save memory.
3. Return the decoded text.

We’ll also support a very lightweight streaming mode by yielding tokens one by one.


In [ ]:
from typing import AsyncGenerator

@app.post("/chat")
async def chat(request: ChatRequest):
    """Return a generated response for the given prompt.

    Parameters
    ----------
    request.prompt: str
        The user prompt.
    request.max_new_tokens: int, optional
        How many new tokens to generate.
    """
    # Encode prompt
    input_ids = tokenizer(request.prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(next(model.parameters()).device)

    # Generation kwargs – tweak as needed
    gen_kwargs = {
        "max_new_tokens": request.max_new_tokens or 150,
        "temperature": 0.7,
        "top_p": 0.9,
        "top_k": 50,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
    }

    with torch.no_grad():
        output_ids = model.generate(input_ids, **gen_kwargs)

    response_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return {"response": response_text}


### Optional: Streaming endpoint

If you want to stream tokens back to the client (e.g., for a chat UI), you can use the following endpoint. It yields one token at a time.


In [ ]:
@app.post("/chat-stream")
async def chat_stream(request: ChatRequest) -> AsyncGenerator[str, None]:
    input_ids = tokenizer(request.prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(next(model.parameters()).device)

    gen_kwargs = {
        "max_new_tokens": request.max_new_tokens or 150,
        "temperature": 0.7,
        "top_p": 0.9,
        "top_k": 50,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
        "streaming": True,  # enable streaming in transformers 4.35
    }

    with torch.no_grad():
        for token_id in model.generate(input_ids, **gen_kwargs):
            token_text = tokenizer.decode(token_id, skip_special_tokens=True)
            yield token_text


## 6️⃣  Run the server locally

```bash
uvicorn main:app --host 0.0.0.0 --port 8000 --reload
```

- `main` is the name of the Python file (e.g., `main.py`).
- `--reload` restarts the server automatically when you edit the code.

You should see a message like:

```
INFO:     Started server process [12345]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
```


## 7️⃣  Test the API with `curl`

```bash
curl -X POST http://localhost:8000/chat \
     -H "Content-Type: application/json" \
     -d '{"prompt": "Hello, TinyLlama! What is the capital of France?"}'
```

You should receive a JSON response:

```json
{"response": "The capital of France is Paris."}
```

For streaming, you can use `curl` with `-N` to see incremental output:

```bash
curl -N -X POST http://localhost:8000/chat-stream \
     -H "Content-Type: application/json" \
     -d '{"prompt": "Tell me a short joke."}'
```


## 8️⃣  Deploying to the Cloud or Docker

Once you’re happy locally, you can:

- **Dockerize** the app:
  ```dockerfile
  FROM python:3.10-slim
  WORKDIR /app
  COPY requirements.txt .
  RUN pip install -r requirements.txt
  COPY . .
  CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]
  ```
- **Deploy** to a cloud platform (e.g., AWS Lambda via AWS‑SAM, Google Cloud Run, or Azure Functions). FastAPI is fully compatible with ASGI servers, so the transition is smooth.

💡 **Tip**: Use environment variables to pass the `HF_TOKEN` securely in production.


## Knowledge Check

Test your understanding with these questions:


### Question 1

Which library is required to load GGUF models in Hugging Face Transformers?

A. torch
B. transformers
C. accelerate
D. huggingface_hub

**Answer:** B

**Explanation:** The `transformers` library provides the `AutoModelForCausalLM.from_pretrained` method that supports GGUF format via the `trust_remote_code=True` flag.
